In [1]:
def set_paths():
    '''
    Deployment should be something like "dirs/dir1/use-cases"
    This function adds to the path "dirs/dir1/use-cases" and "dirs/dir1/"
    :return:
    '''
    import imp
    from os.path import dirname
    import os
    import sys

    USE_CASES = "/var/SP/data/home/bgmerin1/src/Repositorios/use-cases"#dirname(os.path.abspath(imp.find_module('churn')[1]))

    if USE_CASES not in sys.path:
        sys.path.append(USE_CASES)
        print("Added '{}' to path".format(USE_CASES))

    # if deployment is correct, this path should be the one that contains "use-cases", "pykhaos", ...
    # FIXME another way of doing it more general?
    DEVEL_SRC = os.path.dirname(USE_CASES)  # dir before use-cases dir
    if DEVEL_SRC not in sys.path:
        sys.path.append(DEVEL_SRC)
        print("Added '{}' to path".format(DEVEL_SRC))
set_paths()

Added '/var/SP/data/home/bgmerin1/src/Repositorios/use-cases' to path
Added '/var/SP/data/home/bgmerin1/src/Repositorios' to path


In [2]:
# coding=utf-8

import sys

from common.src.main.python.utils.hdfs_generic import *
import argparse
import os
import sys
import time
from pyspark.sql.functions import (udf,
                                    col,
                                    decode,
                                    when,
                                    lit,
                                    lower,
                                    concat,
                                    translate,
                                    count,
                                    sum as sql_sum,
                                    max as sql_max,
                                    min as sql_min,
                                    avg as sql_avg,
                                    greatest,
                                    least,
                                    isnull,
                                    isnan,
                                    struct, 
                                    substring,
                                    size,
                                    length,
                                    year,
                                    month,
                                    dayofmonth,
                                    unix_timestamp,
                                    date_format,
                                    from_unixtime,
                                    datediff,
                                    to_date, 
                                    desc,
                                    asc,
                                    countDistinct,
                                    row_number,
                                    skewness,
                                    kurtosis,
                                    concat_ws)

from pyspark.sql import Row, DataFrame, Column, Window
from pyspark.sql.types import DoubleType, StringType, IntegerType, DateType, ArrayType, FloatType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from datetime import datetime
from itertools import chain
import numpy as np
from functools import reduce
from churn.models.fbb_churn_amdocs.utils_general import *
from pykhaos.utils.date_functions import convert_to_date
from churn.models.fbb_churn_amdocs.utils_model import *
from churn.models.fbb_churn_amdocs.utils_fbb_churn import *
from churn.models.fbb_churn_amdocs.metadata_fbb_churn import *
from churn.models.fbb_churn_amdocs.feature_selection_utils import *
import subprocess

In [3]:
# Para que no haga falta recargar el notebook si hacemos un cambio en alguna de las funciones que estamos importando
%load_ext autoreload
%autoreload 2

In [4]:
# set BDP parameters
def setting_bdp(min_n_executors = 3, max_n_executors = 15, n_cores = 8, executor_memory = "12g", driver_memory="8g",
                   app_name = "Python app", driver_overhead="1g", executor_overhead='3g'):

    MAX_N_EXECUTORS = max_n_executors
    MIN_N_EXECUTORS = min_n_executors
    N_CORES_EXECUTOR = n_cores
    EXECUTOR_IDLE_MAX_TIME = 120
    EXECUTOR_MEMORY = executor_memory
    DRIVER_MEMORY = driver_memory
    N_CORES_DRIVER = 1
    MEMORY_OVERHEAD = N_CORES_EXECUTOR * 2048
    QUEUE = "root.BDPtenants.es.medium"
    BDA_CORE_VERSION = "1.0.0"

    SPARK_COMMON_OPTS = os.environ.get('SPARK_COMMON_OPTS', '')
    SPARK_COMMON_OPTS += " --executor-memory %s --driver-memory %s" % (EXECUTOR_MEMORY, DRIVER_MEMORY)
    SPARK_COMMON_OPTS += " --conf spark.shuffle.manager=tungsten-sort"
    SPARK_COMMON_OPTS += "  --queue %s" % QUEUE

    # Dynamic allocation configuration
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.shuffle.service.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.maxExecutors=%s" % (MAX_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.minExecutors=%s" % (MIN_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.executor.cores=%s" % (N_CORES_EXECUTOR)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.executorIdleTimeout=%s" % (EXECUTOR_IDLE_MAX_TIME)
    # SPARK_COMMON_OPTS += " --conf spark.ui.port=58235"
    SPARK_COMMON_OPTS += " --conf spark.port.maxRetries=100"
    SPARK_COMMON_OPTS += " --conf spark.app.name='%s'" % (app_name)
    SPARK_COMMON_OPTS += " --conf spark.submit.deployMode=client"
    SPARK_COMMON_OPTS += " --conf spark.ui.showConsoleProgress=true"
    SPARK_COMMON_OPTS += " --conf spark.sql.broadcastTimeout=1200"
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.memoryOverhead={}".format(executor_overhead)
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.driverOverhead={}".format(driver_overhead)

    BDA_ENV = os.environ.get('BDA_USER_HOME', '')

    # Attach bda-core-ra codebase
    SPARK_COMMON_OPTS+=" --files {}/scripts/properties/red_agent/nodes.properties,{}/scripts/properties/red_agent/nodes-de.properties,{}/scripts/properties/red_agent/nodes-es.properties,{}/scripts/properties/red_agent/nodes-ie.properties,{}/scripts/properties/red_agent/nodes-it.properties,{}/scripts/properties/red_agent/nodes-pt.properties,{}/scripts/properties/red_agent/nodes-uk.properties".format(*[BDA_ENV]*7)

    os.environ["SPARK_COMMON_OPTS"] = SPARK_COMMON_OPTS
    os.environ["PYSPARK_SUBMIT_ARGS"] = "%s pyspark-shell " % SPARK_COMMON_OPTS
   

In [5]:
def get_spark_session(app_name="default name", log_level='INFO', min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    HOME_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src")
    if HOME_SRC not in sys.path:
        sys.path.append(HOME_SRC)


    setting_bdp(app_name=app_name, min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores, executor_memory = executor_memory, driver_memory=driver_memory)
    from common.src.main.python.utils.hdfs_generic import run_sc
    sc, spark, sql_context = run_sc(log_level=log_level)


    return sc, spark, sql_context

In [6]:
def initialize(app_name, min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    import time
    start_time = time.time()

    print("_initialize spark")
    #import pykhaos.utils.pyspark_configuration as pyspark_config
    sc, spark, sql_context = get_spark_session(app_name=app_name, log_level="OFF", min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores,
                             executor_memory = executor_memory, driver_memory=driver_memory)
    print("Ended spark session: {} secs | default parallelism={}".format(time.time() - start_time,
                                                                         sc.defaultParallelism))
    return spark

In [7]:
spark = initialize("PBMA Incrementales ",executor_memory = "32g",min_n_executors = 6,max_n_executors = 15)


_initialize spark
Ended spark session: 35.7094120979 secs | default parallelism=2


In [ ]:
ciclo_ini='20190521'
horizon=2
nombre='pbma_incremental_'+ciclo_ini

In [ ]:
dfPBMA_ini = (spark.read.parquet('/data/udf/vf_es/churn/extra_feats_mod/pbma_srv/year={}/month={}/day={}'
    .format(int(ciclo_ini[0:4]), int(ciclo_ini[4:6]), int(ciclo_ini[6:8]))))


In [ ]:
# Calculamos n ciclos hacia atrás (donde n es el horizon)
cycle = 0
fini_tmp = ciclo_ini
fechas=[]
while cycle < horizon:
    yearmonthday_target = get_previous_cycle(fini_tmp, str_fmt="%Y%m%d")
    cycle = cycle + 1
    fini_tmp = yearmonthday_target
    fechas.append(fini_tmp)
ciclo_prev=fini_tmp

In [ ]:
ciclo_prev

In [ ]:
print(fechas)

In [ ]:
colResta=[]
variables_ident=['msisdn','msisdn_d','num_cliente','num_cliente_d','nif_cliente_d','nif_cliente']

for ccol in dfPBMA_ini.columns:
    if ccol not in variables_ident:
        
        colResta.append(ccol)
        dfPBMA_ini=dfPBMA_ini.withColumnRenamed(ccol,ccol+'_ini')

In [ ]:
cont=1

dfPBMA_ini.select(['msisdn','pbms_srv_ind_soporte_ini']).filter(col('msisdn')=='941177042').show()

for ff in fechas:
    
    print(ff)
    
    dfPBMA_prev = (spark.read.parquet('/data/udf/vf_es/churn/extra_feats_mod/pbma_srv/year={}/month={}/day={}'
    .format(int(ff[0:4]), int(ff[4:6]), int(ff[6:8]))))
       
    for ccol in dfPBMA_prev.columns:
        if ccol not in variables_ident:
            dfPBMA_prev=dfPBMA_prev.withColumnRenamed(ccol,ccol+'_prev')
            
    dfPBMA_prev.select(['msisdn','pbms_srv_ind_soporte_prev']).filter(col('msisdn')=='941177042').show()
            
    dfResta_tmp=dfPBMA_ini.join(dfPBMA_prev,on=variables_ident,how='left')       
      
    for ccol in colResta:
    
        dfResta_tmp=dfResta_tmp.withColumn(ccol+'_dif_w'+str(cont),col(ccol+'_ini')-col(ccol+'_prev'))
        
        dfResta_tmp=dfResta_tmp.drop(ccol+'_ini')
        dfResta_tmp=dfResta_tmp.drop(ccol+'_prev')
                
    if cont==1: dfResta=dfResta_tmp
    else: dfResta=dfResta.join(dfResta_tmp,on=variables_ident,how='left')
            
    print(cont,dfPBMA_ini.count(),dfResta_tmp.count(),dfResta.count(),dfPBMA_prev.count())

    cont=cont+1

dfResta.select(['msisdn','pbms_srv_ind_soporte_dif_w1','pbms_srv_ind_soporte_dif_w2']).filter(col('msisdn')=='941177042').show()


In [ ]:
dfRestaFin=(dfResta.withColumn('year',lit(int(ciclo_ini[0:4])))
.withColumn('month',lit(int(ciclo_ini[4:6])))
.withColumn('day',lit(int(ciclo_ini[6:8]))))



In [ ]:
(dfRestaFin.write.partitionBy('year', 'month', 'day').mode("append").format("parquet")
            .save('/data/attributes/vf_es/trigger_analysis/pbma_incremental/'))

In [ ]:
#dfResta.repartition(200).write.save('/data/attributes/vf_es/trigger_analysis/'+nombre, format='parquet', mode='overwrite')


In [64]:
yearmonthday ='20190314'# convert_to_date('20181130')
yearmonthday_target = '20190414' #convert_to_date('20181231')

In [65]:
#fixport = getFixPortRequestsForCycleList(spark, yearmonthday, yearmonthday_target)


20190314
20190414
[Info getFixPortRequestsForMonth] Fri Jun 21 09:56:56 2019 Port-out requests for fixed services during period 20190314-20190414: 54000


In [97]:
fixport = spark.read.table("raw_es.portabilitiesinout_portafijo") \
        .filter(col("INICIO_RANGO") == col("FIN_RANGO")) \
        .withColumn("label_srv", lit(1.0)) \
        .where(
        (col('FECHA_INSERCION_SGP') >= yearmonthday_obj) & (col('FECHA_INSERCION_SGP') <= yearmonthday_target_obj))


In [43]:
fixport.columns

['msisdn_d', 'FECHA_INSERCION_SGP', 'label_srv']

In [66]:
fixdxtr = getFbbDxsForCycleList(spark,yearmonthday, yearmonthday_target)


[Info getFbbDxsForMonth] Fri Jun 21 09:57:10 2019 DXs for FBB services during the period: 20190314-20190414: 67487


In [68]:
origin = '/user/hive/warehouse/tests_es.db/jvmm_amdocs_ids_'
trfeatdf = getCarNumClienteDf(spark, origin, '20190314')

[Info getCarNumClienteDf] Fri Jun 21 10:11:22 2019 Size of the original df for 20190314: 15301817
[Info FbbChurn] Fri Jun 21 10:19:55 2019 Saving origindf_tmp to HDFS
[Info getCarNumClienteDf] Fri Jun 21 10:19:56 2019 Size of the origindf after load from HDFS: 15301817
[Info getCarNumClienteDf] Fri Jun 21 10:21:27 2019 Size of the numclidf for 20190314 - Num rows: 2337071 - Num columns: 101
[Info FbbChurn] Fri Jun 21 10:26:41 2019 Saving numclidf_tmp to HDFS
[Info get_billing_df] Fri Jun 21 10:26:42 2019 Starting the preparation of billing feats
[Info get_billing_df] Fri Jun 21 10:27:46 2019 Size of the df with billing feats: 6207643
[Info getCarNumClienteDf] Fri Jun 21 10:28:13 2019 Size of the tmpdf for 20190314 - Num rows: 2337071 - Num columns: 134
[Info get_mobile_spinners_df] Fri Jun 21 10:28:45 2019 Port-out data loaded for 20190314 with a total of 28960500 rows and 11643657 distinct NIFs
[Info get_mobile_spinners_df] Fri Jun 21 10:29:31 2019 Timing feats computed for a total of

In [69]:
window = Window.partitionBy("num_cliente")
selcols = getIdFeats() + getCrmFeats() + getBillingFeats() + getMobSopoFeats() + getOrdersFeats()

unbaltrdf_orig = trfeatdf\
    .join(fixport, ['msisdn_d'], "left_outer")\
    .na.fill({'label_srv': 0.0})\
    .join(fixdxtr, ['msisdn'], "left_outer")\
    .na.fill({'label_dx': 0.0})\
    .withColumn('tmp', when((col('label_srv')==1.0) | (col('label_dx')==1.0), 1.0).otherwise(0.0))\
    .withColumn('label', sql_max('tmp').over(window))\
    .filter(col("rgu")=="fbb")\
    .select(selcols + ['label'])

In [70]:
unbaltrdf_orig.groupBy('label').agg(count('*')).show()


+-----+--------+
|label|count(1)|
+-----+--------+
|  0.0| 2269578|
|  1.0|   67493|
+-----+--------+



In [71]:
unbaltrdf_d = trfeatdf.join(fixdxtr, on=['msisdn'], how="inner").filter(col("rgu")=="fbb")\

unbaltrdf_d.count()



67487

In [75]:
unbaltrdf = trfeatdf.join(fixport, on=['msisdn_d'], how="inner").filter(col("rgu")=="fbb")
unbaltrdf.count()

0

In [99]:
fixport.count()

47422

In [109]:
trfeatdf_fbb=trfeatdf.filter(col("rgu")=="fbb")
trfeatdf_fbb.select('nif_cliente').join(fixport.select('NIF_CRM'), on=(fixport['NIF_CRM']==trfeatdf_fbb['nif_cliente']), how="inner").count()


188784

In [ ]:
# trfeatdf_fbb ('nif_cliente') y fixport ('NIF_SP'): 5099
# trfeatdf_fbb ('nif_cliente') y fixport ('NIF_CRM'): 188784
# trfeatdf_fbb ('num_cliente') y fixport ('NUM_CLIENTE_SP'): 0


In [83]:
trfeatdf.columns

['num_cliente',
 'nif_cliente',
 'rgu',
 'campo1',
 'msisdn',
 'msisdn_d',
 'mean_dias_desde_fx_srv_basic',
 'movil_services',
 'tv_services',
 'total_penal_cust_pending_n3_penal_amount',
 'x_formato_factura',
 'x_user_twitter',
 'total_penal_cust_pending_n4_penal_amount',
 'dias_desde_bam-movil_fx_first',
 'max_dias_desde_fx_trybuy_autom_tv',
 'total_price_tariff',
 'flg_robinson',
 'pvr_services',
 'num_tariff_redl',
 'dias_desde_fx_fbb_upgrade',
 'max_dias_desde_fx_srv_basic',
 'total_price_trybuy',
 'total_penal_srv_pending_n4_penal_amount',
 'max_dias_desde_fx_pvr_tv',
 'max_dias_desde_fx_football_tv',
 'total_max_dias_hasta_penal_cust_pending_end_date',
 'trybuy_services',
 'min_price_srv_basic',
 'total_penal_cust_pending_n2_penal_amount',
 'x_datos_trafico',
 'total_min_dias_hasta_penal_srv_pending_end_date',
 'cta_correo',
 'gender2hgbst_elm',
 'dias_desde_fixed_fx_first',
 'max_price_srv_basic',
 'total_penal_srv_pending_n5_penal_amount',
 'prepaid_services',
 'hz_services',


In [78]:
fixport.columns

['ID_INTERNO',
 'OT_CRM',
 'TICKET_CRM',
 'ID_PROCESO',
 'TIPO_PROCESO',
 'PROCESO',
 'INICIO_RANGO',
 'FIN_RANGO',
 'OPERADOR_RECEPTOR',
 'OPERADOR_DONANTE',
 'NIF_SP',
 'TIPO_ACCESO',
 'FLAG_NUMERACION_COMPLETA',
 'REVENDEDOR_DONANTE',
 'REVENDEDOR_RECEPTOR',
 'FECHA_VENTANA_CAMBIO',
 'NUM_CLIENTE_SP',
 'NOMBRE_CLIENTE',
 'NIF_CRM',
 'CLASE_CLIENTE',
 'ESTADO_NUMERACION',
 'CUC',
 'SERVICIO',
 'NUMERO_LINEAS_ACTIVAS',
 'FECHA_LIMITE_CANCELACION',
 'RGU',
 'AVERIA_RECLAMACION',
 'IMPORTE_FACTURACION',
 'FECHA_INSERCION_SGP',
 'service_processed_at',
 'service_file_id',
 'year',
 'month',
 'day',
 'label_srv']

In [105]:
fixport.select('ID_INTERNO','OT_CRM','ID_PROCESO','INICIO_RANGO','FIN_RANGO','NUM_CLIENTE_SP').show()

+----------+------+--------------------+------------+---------+--------------+
|ID_INTERNO|OT_CRM|          ID_PROCESO|INICIO_RANGO|FIN_RANGO|NUM_CLIENTE_SP|
+----------+------+--------------------+------------+---------+--------------+
|   3974337|      |00001201812031601569|   981610769|981610769|              |
|   3973450|      |00031201812031501043|   828662173|828662173|              |
|   3972707|      |00003201812031502924|   963211365|963211365|              |
|   3972018|      |00031201812031500154|   935380316|935380316|              |
|   3972643|      |00001201812031602098|   912243693|912243693|              |
|   3972534|      |00001201812031601678|   950301429|950301429|              |
|   3973900|      |00003201812031502195|   822102128|822102128|              |
|   3974904|      |00001201812031601313|   942392013|942392013|              |
|   3972548|      |00029201812031501616|   941581556|941581556|              |
|   3971950|      |00031201812031501792|   926036932

In [114]:
trfeatdf.filter(col('rgu')=='fixed').select('num_cliente','campo1','msisdn','msisdn_d').distinct().show()

+-----------+------+------+--------+
|num_cliente|campo1|msisdn|msisdn_d|
+-----------+------+------+--------+
+-----------+------+------+--------+



In [115]:
trfeatdf.filter(col('rgu')=='fbb').select('num_cliente','campo1','msisdn','msisdn_d').distinct().show()

+-----------+---------+--------------+------------+
|num_cliente|   campo1|        msisdn|    msisdn_d|
+-----------+---------+--------------+------------+
|  000066035|     null|  10C29V637EV8|        null|
|  000128808|852613891|  58237FKW8GLT|58238CCE0DEB|
|  000325467|104836096|  8029729G4D0N|8029946D8D5A|
|  000425794|     null|09660072067732|        null|
|  000465856|     null|  111042475320|        null|
|  000509943|648388464|  DC533I253621|DC537C310869|
|  000598367|764379761|  C4044055E076|C4041532D090|
|  000634118|     null|  V33081HTP282|        null|
|  000645564|101492925|  741028868038|741021823652|
|  001148722|700613231|  5C358HV7942S|5C353BC8449B|
|  001488857|     null|  A408R8IV4154|        null|
|  001492800|012816736|  527044974301|527023111951|
|  001519838|025910151|  V48072COV315|V48082AAH532|
|  001551617|014177397|  10C20J917884|10C25A465255|
|  001617790|028665938|  J26067LUG319|J26082AAB207|
|  002057582|845088151|  O48009QUA216|O48082AAF917|
|  002107928

In [116]:
trfeatdf.groupBy('rgu').agg(count('*')).show()

+---+--------+
|rgu|count(1)|
+---+--------+
|fbb| 2337071|
+---+--------+



In [117]:
fixport.groupBy('rgu').agg(count('*')).show()

+---+--------+
|rgu|count(1)|
+---+--------+
|2.0|    8100|
|3.0|   15870|
|1.0|    1416|
|4.0|    6815|
|0.0|   15221|
+---+--------+



In [ ]:
union = trfeatdf.join(fixport, on=['msisdn_d'], how="inner").filter(col("rgu")=="fbb")


In [18]:
trfeatdf.columns

['num_cliente',
 'nif_cliente',
 'rgu',
 'campo1',
 'msisdn',
 'msisdn_d',
 'mean_dias_desde_fx_srv_basic',
 'movil_services',
 'tv_services',
 'total_penal_cust_pending_n3_penal_amount',
 'x_formato_factura',
 'x_user_twitter',
 'total_penal_cust_pending_n4_penal_amount',
 'dias_desde_bam-movil_fx_first',
 'max_dias_desde_fx_trybuy_autom_tv',
 'total_price_tariff',
 'flg_robinson',
 'pvr_services',
 'num_tariff_redl',
 'dias_desde_fx_fbb_upgrade',
 'max_dias_desde_fx_srv_basic',
 'total_price_trybuy',
 'total_penal_srv_pending_n4_penal_amount',
 'max_dias_desde_fx_pvr_tv',
 'max_dias_desde_fx_football_tv',
 'total_max_dias_hasta_penal_cust_pending_end_date',
 'trybuy_services',
 'min_price_srv_basic',
 'total_penal_cust_pending_n2_penal_amount',
 'x_datos_trafico',
 'total_min_dias_hasta_penal_srv_pending_end_date',
 'cta_correo',
 'gender2hgbst_elm',
 'dias_desde_fixed_fx_first',
 'max_price_srv_basic',
 'total_penal_srv_pending_n5_penal_amount',
 'prepaid_services',
 'hz_services',


In [19]:
fixport.columns

['msisdn_d', 'FECHA_INSERCION_SGP', 'label_srv']

In [13]:
trfeatdf.count()

1978186

In [14]:
fixport.count()

47422

In [ ]:
unbaltrdf.columns

In [21]:
trfeatdf.select(['campo1','msisdn','msisdn_d']).show()

+---------+------------+------------+
|   campo1|      msisdn|    msisdn_d|
+---------+------------+------------+
|852613891|58237FKW8GLT|58238CCE0DEB|
|104836096|8029729G4D0N|8029946D8D5A|
|648388464|DC533I253621|DC537C310869|
|764379761|C4044055E076|C4041532D090|
|105186998|V33081HTP282|V33082ACE137|
|101492925|741028868038|741021823652|
|290973451|5C350B82S3O5|5C353B52E0E1|
|187263681|5C354CRR4C91|5C353BDC4C89|
|700613231|5C358HV7942S|5C353BC8449B|
|012816736|527044974301|527023111951|
|025910151|V48072COV315|V48082AAH532|
|014177397|10C20J917884|10C25A465255|
|028665938|J26067LUG319|J26082AAB207|
|845088151|O48009QUA216|O48082AAF917|
|103919077|308084467854|308024460452|
|252678302|DC537N1P86E0|DC537C4B21B5|
|027483321|DC531F687UED|DC537C871BFD|
|105740209|E8ADV43O4495|E8ADA60A2978|
|104595979|O08016XPP363|O08082AFJ075|
|002410800|A408L5WNO863|A408F5DDB018|
+---------+------------+------------+
only showing top 20 rows



In [20]:
fixport.show()

+---------+-------------------+---------+
| msisdn_d|FECHA_INSERCION_SGP|label_srv|
+---------+-------------------+---------+
|968089794|2018-12-11 00:00:00|      1.0|
|952677232|2018-12-11 00:00:00|      1.0|
|976573398|2018-12-11 00:00:00|      1.0|
|949493201|2018-12-11 00:00:00|      1.0|
|977731469|2018-12-11 00:00:00|      1.0|
|971477546|2018-12-11 00:00:00|      1.0|
|918051059|2018-12-11 00:00:00|      1.0|
|959153255|2018-12-11 00:00:00|      1.0|
|938465732|2018-12-11 00:00:00|      1.0|
|935171604|2018-12-11 00:00:00|      1.0|
|954031865|2018-12-11 00:00:00|      1.0|
|961474353|2018-12-11 00:00:00|      1.0|
|935623058|2018-12-11 00:00:00|      1.0|
|916619351|2018-12-11 00:00:00|      1.0|
|952700184|2018-12-11 00:00:00|      1.0|
|945223600|2018-12-11 00:00:00|      1.0|
|955115110|2018-12-31 00:00:00|      1.0|
|916481072|2018-12-31 00:00:00|      1.0|
|979852900|2018-12-31 00:00:00|      1.0|
|986431476|2018-12-31 00:00:00|      1.0|
+---------+-------------------+---